In [ ]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

# Configure make file for gpu usage
%cd /content/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

# Compile
!make

In [ ]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3-tiny.cfg cfg/yolov3_training.cfg

# Change lines in cfg to reference our batches
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '127 s@filters=255@filters=30@' cfg/yolov3_training.cfg
!sed -i '171 s@filters=255@filters=30@' cfg/yolov3_training.cfg
!sed -i '135 s@classes=80@classes=5@' cfg/yolov3_training.cfg
!sed -i '177 s@classes=80@classes=5@' cfg/yolov3_training.cfg



#create an obj file containing the classes
!echo -e 'Tin\nIron\nAdamant\nmith\ncoal' > /content/darknet/data/obj.names
!echo -e 'classes= 5\ntrain  = /content/darknet/data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /content/gdrive/MyDrive/yolov3' > /content/darknet/data/obj.data

In [ ]:
!cp cfg/yolov3_training.cfg /content/gdrive/MyDrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /content/gdrive/MyDrive/yolov3/classes.txt
!mkdir data/obj
!unzip /mydrive/yolov3/images.zip -d data/obj
import glob
images_list = glob.glob("data/obj/images/yolov3/*.png")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
!./darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/yolov3_training.cfg darknet53.conv.74 -dont_show >> /content/gdrive/MyDrive/yolov3/aggregate-voc-tiny7.log

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

!./darknet detector test /content/darknet/data/obj.data cfg/yolov3_training.cfg /content/gdrive/MyDrive/yolov3/yolov3_training_last.weights /content/darknet/data/obj/2021-12-09 12-02-51.mkv

  imShow('predictions.jpg')